In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
from typing import Tuple

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# main flow
start_time = dt.datetime.now()
print("Started at ", start_time)

Started at  2021-01-31 22:11:20.517325


In [3]:
in_kaggle = False


def get_data_file_path(is_in_kaggle: bool) -> Tuple[str, str, str]:
    train_path = ''
    test_path = ''
    sample_submission_path = ''

    if is_in_kaggle:
        # running in Kaggle, inside the competition
        train_path = '../input/tabular-playground-series-jan-2021/train.csv'
        test_path = '../input/tabular-playground-series-jan-2021/test.csv'
        sample_submission_path = '../input/tabular-playground-series-jan-2021/sample_submission.csv'
    else:
        # running locally
        train_path = 'data/train.csv'
        test_path = 'data/test.csv'
        sample_submission_path = 'data/sample_submission.csv'

    return train_path, test_path, sample_submission_path

In [4]:
%%time
# get the training set and labels
train_set_path, test_set_path, sample_subm_path = get_data_file_path(in_kaggle)

df_train = pd.read_csv(train_set_path)
df_test = pd.read_csv(test_set_path)

subm = pd.read_csv(sample_subm_path)

# list of basic raw features
feature_list = [col for col in df_train.columns if col.startswith('cont')]

Wall time: 1.35 s


In [5]:
%%time
# do the train-test split for the model evaluation purpose
from sklearn.model_selection import train_test_split
X = df_train[feature_list] 
X_test = df_test[feature_list] 
y = df_train['target']
# separate into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(210000, 14) (90000, 14) (210000,) (90000,)
Wall time: 1.74 s


In [6]:
from autokeras import StructuredDataRegressor
# define the search
# max 15 models to try
search = StructuredDataRegressor(max_trials=29, loss='mean_squared_error')

# perform the search
search.fit(x=X_train, y=y_train, verbose=1)
# evaluate the model
mse, _ = search.evaluate(X_val, y_val, verbose=1)
print('MSE: %.3f' % mse)

Trial 14 Complete [00h 07m 14s]
val_loss: 0.5114156603813171

Best val_loss So Far: 0.5075059533119202
Total elapsed time: 01h 40m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/67
6563/6563 [==============================] - 7s 1ms/step - loss: 5.0338 - mean_squared_error: 5.0338
Epoch 2/67
6563/6563 [==============================] - 7s 1ms/step - loss: 1.0593 - mean_squared_error: 1.0593
Epoch 3/67
6563/6563 [==============================] - 7s 1ms/step - loss: 0.6733 - mean_squared_error: 0.6733
Epoch 4/67
6563/6563 [==============================] - 7s 1ms/step - loss: 0.5500 - mean_squared_error: 0.5500
Epoch 5/67
6563/6563 [==============================] - 7s 1ms/step - loss: 0.5247 - mean_squared_error: 0.5247
Epoch 6/67
6563/6563 [==============================] - 7s 1ms/step - loss: 0.5196 - mean_squared_error: 0.5196
Epoch 7/67
6563/6563 [==============================] - 7s 1ms/step - loss: 0.5182 - mean_squared_error: 0.5182
Epoch 8/67
6563/6563 [=====================

In [7]:
# predict with the best model on the test set
yhat = search.predict(X_test)


In [8]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file
model.save('model_insurance.h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 14)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 14)                0         
_________________________________________________________________
normalization (Normalization (None, 14)                29        
_________________________________________________________________
dense (Dense)                (None, 32)                480       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

In [9]:
# package the final submission file
subm['target'] = yhat
subm.to_csv('autokeras_automl_submission.csv', index=False)

# Submission results

## Best Model out of 15 Trials

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
input_1 (InputLayer)         (None, 14)             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 14)                0         
_________________________________________________________________
normalization (Normalization (None, 14)                29        
_________________________________________________________________
dense (Dense)                (None, 32)                480       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0         
_________________________________________________________________
regression_head_1 (Dense)    (None, 1)                 33        
_________________________________________________________________
Total params: 1,598
Trainable params: 1,569
Non-trainable params: 29

- RMSE on the public leaderboard: 0.71698
- MSE on the local validation: 0.511
- RMSE on the local validation: 0.714947551
- Training time: 01h 5m 14s

## Submission 2

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
input_1 (InputLayer)          (None, 14)               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 14)                0         
_________________________________________________________________
normalization (Normalization (None, 14)                29        
_________________________________________________________________
dense (Dense)                (None, 32)                480       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
regression_head_1 (Dense)    (None, 1)                 33        

Total params: 1,598
Trainable params: 1,569
Non-trainable params: 29


- RMSE on the public leaderboard: 0.71490
- MSE on the local validation: 0.509
- RMSE on the local validation: 0,712393852
- Training time: 01h 40m 35s


In [10]:
print('We are done. That is all, folks!')
finish_time = dt.datetime.now()
print("Finished at ", finish_time)
elapsed = finish_time - start_time
print("Elapsed time: ", elapsed)

We are done. That is all, folks!
Finished at  2021-02-01 00:00:25.203076
Elapsed time:  1:49:04.685751
